원본 + 4컷 ->벡터 추출 / test는 원본으로만

In [42]:
import numpy as np
import pandas as pd
import cv2 
from PIL import Image
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.linear_model import LogisticRegression

In [43]:
y = np.load('new_y.npy')
x = np.load('new_cut_feature.npy')
x_ag = np.load('new_cut_4cutting_feature.npy')
x.shape , y.shape , x_ag.shape

((1900, 138), (1900,), (7600, 138))

In [44]:
x_ar = []
x_en = []

y_ar = []
y_en = []
for i,v in enumerate(x):
    if i%4<2:
        x_ar.append(v)
        y_ar.append(y[i])
    else:
        x_en.append(v)
        y_en.append(y[i])
        
x_ar , y_ar , x_en , y_en= np.array(x_ar),np.array(y_ar),np.array(x_en),np.array(y_en)

print(x_ar.shape , x_en.shape , y_ar.shape , y_en.shape)

(950, 138) (950, 138) (950,) (950,)


In [45]:
#train test = 282 : 193

#아랍 데이터 나누기
x_train_ar , x_test_ar = x_ar[:564] , x_ar[564:]
y_train_ar , y_test_ar = y_ar[:564] , y_ar[564:]


#영어 데이터 나누기
x_train_en , x_test_en = x_en[:564] , x_en[564:]
y_train_en , y_test_en = y_en[:564] , y_en[564:]


#영아 + 아랍 데이터
x_train_all = np.concatenate((x_train_ar,x_train_en),axis=0)
x_test_all =  np.concatenate((x_test_ar,x_test_en),axis=0)

y_train_all = np.concatenate((y_train_ar,y_train_en),axis=0)
y_test_all = np.concatenate((y_test_ar,y_test_en),axis=0)

print(x_train_ar.shape , x_test_ar.shape)
print(y_train_ar.shape , y_test_ar.shape)

print(x_train_en.shape , x_test_en.shape)
print(y_train_en.shape , y_test_en.shape)
      
print(x_train_all.shape , x_test_all.shape)
print(y_train_all.shape , y_test_all.shape)

(564, 138) (386, 138)
(564,) (386,)
(564, 138) (386, 138)
(564,) (386,)
(1128, 138) (772, 138)
(1128,) (772,)


In [46]:
#데이터 증강 나누기
x_ag_ar , x_ag_en = [] , []
y_ag_ar , y_ag_en = [] , []

for i , v in enumerate(x_ag[:4512]):
    idx = i//4 #몇번째 사람인지
    if i%16 < 8:
        x_ag_ar.append(v)
        y_ag_ar.append(y[idx])
    else:
        x_ag_en.append(v)
        y_ag_en.append(y[idx])
        
x_ag_ar , x_ag_en = np.array(x_ag_ar) , np.array(x_ag_en)
y_ag_ar , y_ag_en = np.array(y_ag_ar) , np.array(y_ag_en)
print(x_ag_ar.shape , x_ag_en.shape)
print(y_ag_ar.shape , y_ag_en.shape)

(2256, 138) (2256, 138)
(2256,) (2256,)


In [47]:
#원본 + 증강 합치기
x_train_ar = np.concatenate((x_train_ar,x_ag_ar),axis=0)
y_train_ar = np.concatenate((y_train_ar,y_ag_ar),axis=0)

x_train_en = np.concatenate((x_train_en,x_ag_en),axis=0)
y_train_en = np.concatenate((y_train_en,y_ag_en),axis=0)

x_train_all = np.concatenate((x_train_ar,x_train_en),axis=0)
y_train_all = np.concatenate((y_train_ar,y_train_en),axis=0)

print(x_train_ar.shape , x_test_ar.shape)
print(y_train_ar.shape , y_test_ar.shape)

print(x_train_en.shape , x_test_en.shape)
print(y_train_en.shape , y_test_en.shape)
      
print(x_train_all.shape , x_test_all.shape)
print(y_train_all.shape , y_test_all.shape)

(2820, 138) (386, 138)
(2820,) (386,)
(2820, 138) (386, 138)
(2820,) (386,)
(5640, 138) (772, 138)
(5640,) (772,)


In [112]:
#Case마다 여기서 변경
train_x = x_train_ar
test_x = x_test_ar

train_y = y_train_ar
test_y = y_test_ar

cs = StandardScaler()
train_x = cs.fit_transform(train_x)
test_x = cs.transform(test_x)



#모두 False나와야 함
print(np.isnan(train_x).all(),np.isnan(test_x).all(),
      np.isnan(train_y).all(),np.isnan(test_y).all())
#모두 True 나와야 함
print(np.isfinite(train_x).all(),np.isfinite(test_x).all(),
      np.isfinite(train_y).all(),np.isfinite(test_y).all())

print(f'train data length: {len(train_x)} , {len(train_y)}')
print(f'test data length: {len(test_x)} , {len(test_y)}')

False False False False
True True True True
train data length: 2820 , 2820
test data length: 386 , 386


SVM

In [96]:
pd.set_option('display.max.colwidth', 100)

In [97]:
param_grid = {'kernel':['rbf'], 'C':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
              'gamma':['auto','scale',0.0001,0.0005,0.001,0.002,0,0.005,0.007,0.01,0.02,0.05]}

svm_clf = SVC(random_state=100)
grid = GridSearchCV(svm_clf, param_grid=param_grid,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
grid.fit(train_x, train_y)

print(grid.best_score_,grid.best_params_)
print(pd.DataFrame(grid.cv_results_).sort_values(by='rank_test_score')[['rank_test_score','params','mean_test_score']][:5])

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  2.2min finished


0.8815602836879431 {'C': 12, 'gamma': 0.01, 'kernel': 'rbf'}
     rank_test_score                                        params  \
141                1     {'C': 12, 'gamma': 0.01, 'kernel': 'rbf'}   
177                1     {'C': 15, 'gamma': 0.01, 'kernel': 'rbf'}   
157                1  {'C': 14, 'gamma': 'scale', 'kernel': 'rbf'}   
169                1  {'C': 15, 'gamma': 'scale', 'kernel': 'rbf'}   
165                1     {'C': 14, 'gamma': 0.01, 'kernel': 'rbf'}   

     mean_test_score  
141          0.88156  
177          0.88156  
157          0.88156  
169          0.88156  
165          0.88156  


In [113]:
svm_pred = grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, svm_pred)*100, '%')

Accuracy: 71.50259067357513 %


rf

In [105]:
rf_param_grid = {
    'n_estimators' : [300,310,320],
    'max_depth' : [19,20,21],      #default : " None",
    'min_samples_leaf' : [1,2],     #default : 1
    'min_samples_split' : [2,3]  #default : 2
}

rf_clf = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(rf_clf, param_grid=rf_param_grid,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
rf_grid.fit(train_x, train_y)

print(rf_grid.best_score_,rf_grid.best_params_)
rf_df = pd.DataFrame(rf_grid.cv_results_).sort_values(by='rank_test_score')
print(rf_df[['params','mean_test_score']][:5])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.1min finished


0.7624113475177304 {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 320}
                                                                                   params  \
14  {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 320}   
13  {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 310}   
12  {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}   
29  {'max_depth': 21, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 320}   
27  {'max_depth': 21, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 300}   

    mean_test_score  
14         0.762411  
13         0.762057  
12         0.761702  
29         0.759574  
27         0.759220  


In [114]:
rf_pred = rf_grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, rf_pred)*100, '%')

Accuracy: 70.98445595854922 %


XGB

In [101]:
xgb_param={
    'n_estimators' : [310,320],
    'learning_rate' : [0.1,0.15],
    'max_depth' : [7,8,9],
    #'gamma' : [0],
    'colsample_bytree' : [0.7,0.9],
    
}
xgb_clf= xgb.XGBClassifier(random_state=42)#tree_method='gpu_hist', gpu_id=0,
xgb_grid = GridSearchCV(xgb_clf, param_grid=xgb_param,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
xgb_grid.fit(train_x, train_y)

print(xgb_grid.best_score_,xgb_grid.best_params_)
xgb_df = pd.DataFrame(xgb_grid.cv_results_).sort_values(by='rank_test_score')
print(xgb_df[['params','mean_test_score']][:5])

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  2.6min finished


0.7776595744680851 {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 310}
                                                                                   params  \
0    {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 310}   
14   {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 310}   
1    {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 320}   
15   {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 320}   
22  {'colsample_bytree': 0.9, 'learning_rate': 0.15, 'max_depth': 9, 'n_estimators': 310}   

    mean_test_score  
0          0.777660  
14         0.777660  
1          0.777305  
15         0.776950  
22         0.776950  


In [115]:
xgb_pred = xgb_grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, xgb_pred)*100, '%')

Accuracy: 71.24352331606218 %


Logistic regression

In [103]:
param_grid = {'penalty':['l1','l2'], 
              'C':[0.001,0.005,0.01,0.05,0.1,0.3,0.5], 
              'max_iter':[100,150,200,250,300,500]}

log_clf = LogisticRegression(random_state = 42)
log_grid = GridSearchCV(log_clf, param_grid=param_grid,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
log_grid.fit(train_x, train_y)
print(log_grid.best_score_,log_grid.best_params_)
print(pd.DataFrame(log_grid.cv_results_).sort_values(by='rank_test_score')[['rank_test_score','params','mean_test_score']][:5])

Fitting 5 folds for each of 84 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 389 out of 420 | elapsed:    3.4s remaining:    0.2s


0.7375886524822695 {'C': 0.01, 'max_iter': 100, 'penalty': 'l2'}
    rank_test_score                                         params  \
35                1  {'C': 0.01, 'max_iter': 500, 'penalty': 'l2'}   
31                1  {'C': 0.01, 'max_iter': 250, 'penalty': 'l2'}   
29                1  {'C': 0.01, 'max_iter': 200, 'penalty': 'l2'}   
27                1  {'C': 0.01, 'max_iter': 150, 'penalty': 'l2'}   
25                1  {'C': 0.01, 'max_iter': 100, 'penalty': 'l2'}   

    mean_test_score  
35         0.737589  
31         0.737589  
29         0.737589  
27         0.737589  
25         0.737589  


[Parallel(n_jobs=-1)]: Done 420 out of 420 | elapsed:    3.7s finished


In [117]:
log_pred = log_grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, log_pred)*100, '%')

Accuracy: 68.13471502590673 %
